In [ ]:
try:
    from google.colab import drive
    %tensorflow_version 2.x
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

if COLAB:
  !sudo apt-get install -y xvfb ffmpeg
  !pip install -q 'gym==0.10.11'
  !pip install -q 'imageio==2.4.0'
  !pip install -q PILLOW
  !pip install -q 'pyglet==1.3.2'
  !pip install -q pyvirtualdisplay
  !pip install -q --upgrade tensorflow-probability
  !pip install -q tf-agents

  # ROM dependencies for atari games
  ! wget http://www.atarimania.com/roms/Roms.rar
  ! mkdir /content/ROM/
  ! unrar e /content/Roms.rar /content/ROM/
  ! python -m atari_py.import_roms /content/ROM/

  # For visualisation on colab
  !pip install gym pyvirtualdisplay > /dev/null 2>&1
  !apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

  # gym installation
  !apt-get update > /dev/null 2>&1
  !apt-get install cmake > /dev/null 2>&1
  !pip install --upgrade setuptools 2>&1
  !pip install ez_setup > /dev/null 2>&1
  !pip install gym[atari] > /dev/null 2>&1

  # For use of GPU
  %tensorflow_version 2.x
  import tensorflow as tf
  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
    global CPU
    CPU = True
  print('Found GPU at: {}'.format(device_name))

  # Required to save models in HDF5 format
  !pip install pyyaml h5py

Note: using Google CoLab
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
xvfb is already the newest version (2:1.19.6-1ubuntu4.9).
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.
ERROR: tf-agents 0.8.0 has requirement gym>=0.17.0, but you'll have gym 0.10.11 which is incompatible.
ERROR: tf-agents 0.8.0 has requirement gym>=0.17.0, but you'll have gym 0.10.11 which is incompatible.
ERROR: tf-agents 0.8.0 has requirement tensorflow-probability==0.12.2, but you'll have tensorflow-probability 0.13.0 which is incompatible.
--2021-07-15 11:01:01--  http://www.atarimania.com/roms/Roms.rar
Resolving www.atarimania.com (www.atarimania.com)... 195.154.81.199
Connecting to www.atarimania.com (www.atarimania.com)|195.154.81.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11128004 (11M) [application/x-rar-compressed]
Saving to: ‘Roms.rar.2’

Roms.r

--2021-07-15 09:36:35--  http://www.atarimania.com/roms/Roms.rar
Resolving www.atarimania.com (www.atarimania.com)... 195.154.81.199
Connecting to www.atarimania.com (www.atarimania.com)|195.154.81.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11128004 (11M) [application/x-rar-compressed]
Saving to: ‘Roms.rar’

Roms.rar            100%[===================>]  10.61M   836KB/s    in 13s     

2021-07-15 09:36:48 (807 KB/s) - ‘Roms.rar’ saved [11128004/11128004]


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/Roms.rar

Extracting  /content/ROM/HC ROMS.zip                                      36%  OK 
Extracting  /content/ROM/ROMS.zip                                         74% 99%  OK 
All OK
copying adventure.bin from ROMS/Adventure (1980) (Atari, Warren Robinett) (CX2613, CX2613P) (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin fr

Requirement already up-to-date: setuptools in /usr/local/lib/python3.7/dist-packages (57.2.0)


SystemError: ignored

In [91]:
import gym
from gym.wrappers import Monitor
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay
from google.colab import files,drive

display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment 
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  files.download(mp4list[0])
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    print(video)
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [96]:
# env = wrap_env(gym.make("MountainCar-v0"))
# with tf.device('/device:GPU:0'):   

for i in range(3):
  env = wrap_env(gym.make("MsPacman-v0")) 
  observation = env.reset()
  while True:
        
        # env.render()
        #your agent goes here
        action = env.action_space.sample() 
        if(i==1): action = env.action_space.sample()

        observation, reward, done, info = env.step(action)    
            
        if done:
            env.close()
            show_video()
            break;

# show_video()

IndentationError: ignored

%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [58]:
import tensorflow as tf
import numpy as np
import gym
import scipy.misc
import os
import random
import time
import matplotlib.pyplot as plt
from collections import deque
from skimage import transform
from google.colab import files,drive
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.losses import MeanSquaredError 
from tensorflow.keras.metrics import MeanSquaredError
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity


In [41]:
env = wrap_env(gym.make('MsPacman-v0'))
observation = env.reset()
# input_shape = preprocess_observation(observation).shape
input_shape = observation.shape
classes = env.action_space.n
env.close()
# print(input_shape, classes)

In [42]:
def identity_block(X, f, filters, training=True, initializer=random_uniform):

    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    X = Activation('relu')(X)

    ## Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = f, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    X = Activation('relu')(X)

    ## Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    
    ## Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)

    return X

In [43]:
def convolutional_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):

    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    
    # First component of main path glorot_uniform(seed=0)
    X = Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    ## Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = f, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    X = Activation('relu')(X)

    ## Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    
    ##### SHORTCUT PATH ##### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = 1, strides = (s,s), padding = 'valid', kernel_initializer = initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut, training = training) # Default axis
    
    # Final step: 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [44]:
def ResNet50(input_shape, classes): # input and classes depends on gym environment
    """
    Stage-wise implementation of the architecture of the popular ResNet50:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])
    
    ## Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], s = 2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512]) 
    
    ## Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    ## Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    ## AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size = (2,2))(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [45]:
def Deepminds_model(input_shape, classes):
        
        
    X_input = Input(input_shape)
    
    X = Conv2D(32, (8, 8), strides = (4, 4), kernel_initializer = glorot_uniform(seed=0))(X_input)
    X = Activation('relu')(X)
    X = Conv2D(64, (4, 4), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = Activation('relu')(X)
    X = Conv2D(64, (3, 3), strides = (1, 1), kernel_initializer = glorot_uniform(seed=0))(X)
    X = Activation('relu')(X)
    X = Flatten()(X)
    X = Dense(512, activation='relu', kernel_initializer = glorot_uniform(seed=0))(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    model = Model(inputs = X_input, outputs = X)

    return model
    
    

In [46]:
def scale_lumininance(img):
    return np.dot(img[...,:3],[0.299,0.587,0.114])

In [47]:
def preprocess_observation(obs):
    """
    To reduce the computation load we preprocess the date obtained from the atari-game env by
    reducing the resolution and adjusting the color.
    
    DeepMind took the maximum pixel value over subsequent frames to reduce flickering
    caused by the limitations of the Atari platform and then scale it from its current 
    210×160×3 resolution to 84×84.
    
    To convert this, we will take the luminance channel (denoted as Y) from the image, which is the our RGB channel, and apply linear weights to 
    the channel to transform it according to the relative luminance.
    Y = 0.299R + 0.587G + 0.114B
    """
    
    obs_gray = scale_lumininance(obs)
    obs_trans = transform.resize(obs_gray,(84,84))
    return np.moveaxis(obs_trans, 1, 0)

In [48]:
def initialization(input_shape , classes, initial_collect_steps, buffer_size):
    """
    The target_network and prediction_network are initialized as ResNet50/DeepMind's CNN architecture with trainability
    of prediction_network set to False.
    
    The Experience Reply is initialized as a deque whose each element 
    is a tuple (current state, action, reward , done, next state)
    Few actions are executed with the environment to bootstrap the replay data.
    """
    global target_network,prediction_network,experience_replay
#     target_network = ResNet50(input_shape,classes)
    target_network = Deepminds_model(input_shape,classes)  
    # print(target_network.summary())
    
#     prediction_network = ResNet50(input_shape,classes)
    prediction_network = Deepminds_model(input_shape,classes)
    # print(prediction_network.summary())
    
    experience_replay = deque(maxlen = buffer_size)
    
    observation = env.reset()
    for i in range(initial_collect_steps):
        
        # current_state = preprocess_observation(observation)
        current_state = observation
        action = env.action_space.sample()
        observation, reward, done , _ = env.step(action)
        # next_state = preprocess_observation(observation)
        next_state = observation
        # print(i)

        if done:
          observation = env.reset()
        
        sample_observation = (current_state,action,reward,done,next_state)
        experience_replay.append(sample_observation)
    env.close()

In [49]:
def plot(CR , episode, save_fig = True):
    plt.plot([i+1 for i in range(episode)],CR, linewidth=4, label = "DDQN")
    plt.xlabel("Episode")
    plt.ylabel("episode_reward")
    leg = plt.legend(loc='upper left', shadow=True)
    plt.grid()
    ax = plt.gca()
    plt.xlim([0, episode])
    
    output_dir = os.path.join(os.getcwd(), "output")
    if not os.path.exists(output_dir): os.mkdir(output_dir)
    
    if save_fig:
        plt.savefig(os.path.join(output_dir, "plot-{}th-episode.png".format(episode)), bbox_inches="tight")
        files.download(os.path.join(output_dir, "plot-{}th-episode.png".format(episode)))
    else:
        plt.show()
    plt.close()

In [50]:
def eps_greedy_action(Q,eps):
    if np.random.random() < eps: # explore
        return np.random.randint(env.action_space.n)
    else: # exploit
        return np.random.choice(np.flatnonzero(Q == Q.numpy().max()))

In [51]:
# @tf.function
# def train_step(state_minibatch,target,actions):

    

In [64]:
def experiment(N_episodes,gamma,learning_rate, c, batch_size,eps, log_interval):
    
  # Instantiate an optimizer to train the model.
  optimizer = tf.keras.optimizers.RMSprop(learning_rate) # used by deepminds
  # Instantiate a loss function.
  loss_fn = tf.keras.losses.MeanSquaredError()
  # Prepare the metrics.
  train_acc_metric = tf.keras.metrics.MeanSquaredError()
  
  CR = []
  with tf.device('/device:GPU:0'):

    for episode in range(N_episodes):
        start_time = time.time()
        time_step = 0
        env = wrap_env(gym.make('MsPacman-v0'))
        observation = env.reset()
        rewards = [] # compared to total points earned in the game
        while True:
            
            with tf.GradientTape() as tape:
                
                current_state = observation
                # current_state = preprocess_observation(observation) 
                Q = prediction_network(np.array([current_state]), training=False)
                action = eps_greedy_action(Q,eps) 
                new_observation , reward , done , _ = env.step(action)
                rewards.append(reward)
                next_state = new_observation
                # next_state = preprocess_observation(new_observation)
                
                sample_observation = (current_state, action, reward, done, next_state)
                experience_replay.append(sample_observation)
                
                minibatch = random.sample(list(experience_replay),batch_size)
                state_minibatch = np.array([current_state for (current_state, action, reward, done, next_state) in minibatch])
                actions = np.array([action for (current_state, action, reward, done, next_state) in minibatch])
                            
                target = []
                for (current_state, action, reward, done, next_state) in minibatch:
                    if done:
                        target.append(reward)
                    else:
                        Q_target = target_network(np.array([next_state]),training = False)
                        target.append(reward + gamma * Q_target.numpy().max())

                prediction_network_output = prediction_network(state_minibatch, training = True)
                prediction = [Q[action] for Q,action in zip(prediction_network_output,actions)]
                
                loss_value = loss_fn(target , prediction)
                grads = tape.gradient(loss_value, prediction_network.trainable_variables)
                optimizer.apply_gradients(zip(grads, prediction_network.trainable_variables))
                
                # Update training metric.
                train_acc_metric.update_state(target, prediction)

                # S -> S'
                observation = new_observation
                time_step+=1
                if time_step%100 == 0 :
                  print("Time_step:{}".format(time_step))
                
                if episode%c ==0: 
                    target_network.set_weights(prediction_network.get_weights()) 
              
                if done:
                
                    # Display metrics at the end of each episode.
                    train_acc = train_acc_metric.result()
                    print("Training acc over episode: %.4f" % (float(train_acc),))

                    # Reset training metrics at the end of each episode
                    train_acc_metric.reset_states()
                    
                    CR.append(np.squeeze(np.sum(np.array(rewards))))

                    env.close()
                    if (episode%log_interval==0):
                      show_video()
                      output_dir = os.path.join(os.getcwd(), "output")
                      if not os.path.exists(output_dir): os.mkdir(output_dir)
                      prediction_network.save(os.path.join(output_dir,"model-{}.h5".format(episode/log_interval)))
                      files.download(os.path.join(output_dir,"model-{}.h5".format(episode/log_interval)))
                      plot(CR,episode+1)
                    print("Episode {} finished after {} timesteps".format(episode+1, time_step+1))
                    print("{}s taken to finish this episode".format(time.time() - start_time))
                    break

In [65]:
learning_rate = 2.5e-3

log_interval = 100

initial_collect_steps = 200
replay_buffer_max_length = 50000

gamma = 0.99
c = 10 # not sure
eps = 0.00001
batch_size = 32
N_episodes = 12000

initialization(input_shape,classes,initial_collect_steps,replay_buffer_max_length)
experiment(N_episodes, gamma, learning_rate, c, batch_size,eps,log_interval)


Time_step:1
Time_step:2
Time_step:3
Time_step:4
Time_step:5
Time_step:6
Time_step:7
Time_step:8
Time_step:9
Time_step:10
Time_step:11
Time_step:12


KeyboardInterrupt: ignored